In [159]:
import openai
import re
from midiutil.MidiFile import MIDIFile
from pychord import Chord
import random
import pandas as pd

In [160]:
#defining note names for function
NOTES_FLAT = ['C', 'Db', 'D', 'Eb', 'E', 'F', 'Gb', 'G', 'Ab', 'A', 'Bb', 'B']
NOTES_SHARP = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']

def NoteToMidi(KeyOctave):
    # KeyOctave is formatted like 'C#3'
    key = KeyOctave[:-1]  # eg C, Db
    octave = KeyOctave[-1]   # eg 3, 4
    answer = -1

    try:
        if 'b' in key:
            pos = NOTES_FLAT.index(key)
        else:
            pos = NOTES_SHARP.index(key)
    except:
        print('The key is not valid', key)
        return answer

    answer += pos + 12 * (int(octave) + 1) + 1
    return answer

In [161]:
openai.api_key = "REMOVED FOR PRIVACY"
key = 'Bb Minor'
progressions = pd.read_csv('progressions.csv', header=None)
progressions = [list(row) for row in progressions.values]
progression = random.choice(progressions)[0]
format_directions = "Format the response as a python list where each element is 1 chord of the progression.  Do not use roman numerals."
style_prompt = "Generate a {} chord progression in {}.".format(progression, key)
prompt = (f"{style_prompt + ' ' + format_directions}")
prompt.strip()

'Generate a I–iii–vii–IV chord progression in Bb Minor. Format the response as a python list where each element is 1 chord of the progression.  Do not use roman numerals.'

In [162]:
completions = openai.Completion.create(
    engine="text-davinci-003",
    prompt=prompt,
    max_tokens = 1024,
    n=1,
    stop=None,
    temperature=1
)

message = completions.choices[0].text
message = message.replace('\n','')
message
scrape_chords = re.findall(r"\w+\s?\w+",message)
scrape_chords = [elem.replace(" ","") for elem in scrape_chords]
print('Chords: {}'.format(scrape_chords))

chords_notes = []
for chord in scrape_chords:
    chords_notes.append(Chord(chord.replace('Major', '').replace('Minor','m').replace('major','').replace('minor','m').replace('maj','').replace('min','m').replace('Diminished','dim')).components_with_pitch(root_pitch=3))

print('Notes: {}'.format(chords_notes))

chords_notes_midi = [[NoteToMidi(note) for note in sublist] for sublist in chords_notes]
chords_notes_midi = [tuple(sublist) for sublist in chords_notes_midi]

print('MidiData: {}'.format(chords_notes_midi))

Chords: ['BbMinor', 'DbMajor', 'EbMajor', 'AbMajor']
Notes: [['Bb3', 'Db4', 'F4'], ['Db3', 'F3', 'Ab3'], ['Eb3', 'G3', 'Bb3'], ['Ab3', 'C4', 'Eb4']]
MidiData: [(58, 61, 65), (49, 53, 56), (51, 55, 58), (56, 60, 63)]


In [163]:
mf = MIDIFile(1)
track = 0
time = 0
mf.addTrackName(track,time,"Chord Progression")
mf.addTempo(track,time,120)
channel = 0
volume = 100

for chord in chords_notes_midi:
    for note in chord:
        mf.addNote(track,channel,note,time,2,volume)
    time += 2

with open('chords.mid','wb') as output_file:
    mf.writeFile(output_file)

print('midi written to disk')

midi written to disk
